# Models

- Authors: Marc Shapiro, Zeb Engberg
- Date: 2023-04-18
- `pycontrails`: v0.40.1

Model setup, parameterization, and evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/contrailcirrus/2023-04-pycontrails-workshop/blob/main/notebooks/02-Models.ipynb)

In [ ]:
!pip install pycontrails

## ModelParams

Dataclass to prescribe model parameters and defaults.

Generic `ModelParams` contains defaults for interpolation parameters.

In [1]:
import dataclasses

from pycontrails import ModelParams

In [2]:
# implement a ModelParams class with single default parameter `scale`
# generic interpolation defaults added automatically

@dataclasses.dataclass
class MyModelParams(ModelParams):
    """Parameters for :class:`MyModel`."""

    #: Factor
    scale: float = 0.3

MyModelParams().as_dict()

{'copy_source': True,
 'interpolation_method': 'linear',
 'interpolation_bounds_error': False,
 'interpolation_fill_value': nan,
 'interpolation_localize': False,
 'interpolation_use_indices': False,
 'verify_met': True,
 'downselect_met': True,
 'met_longitude_buffer': (0.0, 0.0),
 'met_latitude_buffer': (0.0, 0.0),
 'met_level_buffer': (0.0, 0.0),
 'met_time_buffer': (numpy.timedelta64(0,'h'), numpy.timedelta64(0,'h')),
 'scale': 0.3}

## Model

In [3]:
from typing import Any

import numpy as np
import pandas as pd
import xarray as xr

from pycontrails import Model
from pycontrails import MetDataset, MetDataArray
from pycontrails.core.met_var import AirTemperature

### Definition

Model implementation builds off the `Model` base class.

The following example implements a model that runs on a `MetDataset` grid and returns a `MetDataArray`
that is the `air_temperature` x the `scale` parameter.

In [4]:
class MyModel(Model):

    name = "my-model"
    long_name = "Example model"
    met_variables = (AirTemperature,)  # required met variables
    default_params = MyModelParams  # default ModelParams class
    
    def eval(self, source: MetDataset, **params: Any) -> MetDataArray:
        """My model
        
        Parameters
        ----------
        source : MetDataset
            Dataset defining coordinates to evaluate model.
        **params : Any
            Overwrite model parameters before eval

        Returns
        -------
        MetDataArray
            Returns the variable "air_temperature" x "scale"
        """
        
        return source.data["air_temperature"] * self.params["scale"]

### Initialization

Requires `met` MetDataset

Allows model params to be input as a dictionary to `params` keyword arg.

In [5]:
# create dummy dataset
ds = xr.Dataset(
    {
        "air_temperature": (["longitude", "latitude", "level", "time"], np.ones((20, 15, 4, 5))),
    },
    coords={
        "longitude": np.arange(-100, -80, 1.0),
        "latitude": np.arange(30, 45, 1.0),
        "level": np.arange(100, 500, 100),
        "time": pd.date_range(
                  start="2022-03-01 00:00:00",
                  end="2022-03-01 05:00:00",
                  periods=5),
    },
)
met = MetDataset(ds)

### Evalulation

Requires `source`: Flight / Fleet / MetDataset

In [6]:
model = MyModel(met=met, params={"scale": 0.5})
model.eval(source=met)

<xarray.DataArray 'air_temperature' (longitude: 20, latitude: 15, level: 4,
                                     time: 5)>
array([[[[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        ...,

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
...
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        ...,

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]],

        [[0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5],
         [0.5, 0.5, 0.5, 0.5, 0.5]]]])
Coordinates:
  * longitude     (longitude) float64 -100.0 -99.0 -98.0 ... -83.0 -82.0 -81.0
  * latitude      (latitude) float64 30.0 31.0 32.0 33.0 ... 41.0 42.0 43.0 44.0
  * level         (level) float64 100.0 200.0 300.0 400.0
  * time          (time) datetime64[ns] 2022-03-01 ... 2022-03-01T05:00:00
    air_pressure  (level) float64 1e+04 2e+04 3e+04 4e+04
    altitude      (level) float64 1.618e+04 1.178e+04 9.164e+03 7.185e+03

## Implementations

- [ISSR](02-ISSR.ipynb)
- [SAC](02-SAC.ipynb)
- [CoCiP](02-CoCiP.ipynb)
- [ACCF](02-ACCF.ipynb) (Using https://github.com/dlr-pa/climaccf)

In [ ]:
from pycontrails import Flight

In [ ]:
? Flight

In [ ]:
from pycontrails import Aircraft

In [ ]:
? Aircraft

In [ ]:
filename = "Shanwick_Wypts_2019_01_01.csv"

In [ ]:
def create_flight(flight_id, df_flight):
    """
    Function to create a Flight class from a 
    group of rows with the same Flight ID
    """

    # constant properties along the length of the flight
    flight_attrs = {
        "flight_id": flight_id,
        
        # get first val of atyp col as aircraft type
        "aircraft_type": df_flight["ATYP"].values[0]
    }
    
    # # DISCUSSION: We could use the Aircraft class here
    # aircraft = Aircraft(aircraft_type=df_flight["ATYP"].values[0])
    
    # convert UTC timestamp to np.datetime64
    df_flight["time"] = pd.to_datetime(df_flight["date_time"])

    # get altitude in m
    df_flight["altitude"] = df_flight["alt_ft"] * 0.3048

    # rename a few columns for compatibility with `Flight` requirements
    df_flight = df_flight.rename(columns={"lon_deg": "longitude", "lat_deg": "latitude"})

    # clean up a few columns before building Flight class
    df_flight = df_flight.drop(columns=["Flight ID", "date_time", "ATYP", "alt_ft"])

    # create a Flight class
    return Flight(df_flight, attrs=flight_attrs)

In [ ]:
# read source CSV data
df = pd.read_csv(filename)

# create a list of flights from each Flight ID in dataframe
flights = [
    create_flight(flight_id, df_flight) 
    for (flight_id, df_flight) in df.groupby("Flight ID")
]

In [ ]:
# visualize single flight
flights[10]

In [ ]:
from pycontrails import MetDataset, MetDataArray

In [ ]:
? MetDataset

In [ ]:
? MetDataArray

In [ ]:
from pycontrails.datalib import ERA5, HRES, IFS

In [ ]:
? IFS

In [ ]:
? ERA5

In [ ]:
# get a range of time, use CF names
time_bounds = ("2019-01-01 00:00:00", "2019-01-01 03:00:00")
variables = ["air_temperature", "specific_humidity"]  # supports CF names
era5 = ERA5(time=time_bounds, variables=variables, pressure_levels=[350, 300])
era5

In [ ]:
met = era5.open_metdataset()
met

In [ ]:
# read in Cocip model here to get access to variables
from pycontrails.models import Cocip

print(f"Pressure Level Variables:\n {Cocip.met_variables} \n")
print(f"Single Level Variables:\n {Cocip.rad_variables}")

In [ ]:
time = ("2019-01-01 00:00:00", "2019-01-01 23:00:00")  # (start, end) inclusive time range
pressure_levels = [350, 300, 250, 225, 200, 175, 150]  # select pressure levels

# use pathlib here for cross platform compatibility
# assumes ERA5 data is in the current directory
path_prefix = pathlib.Path(".")

met_filepaths = [
    pathlib.Path(path_prefix / "ERA5_HRES_20190101_Met.nc"),
    pathlib.Path(path_prefix / "ERA5_HRES_20190102_Met.nc"),
]
rad_filepaths = [
    pathlib.Path(path_prefix / "ERA5_HRES_20190101_Rad.nc"),
    pathlib.Path(path_prefix / "ERA5_HRES_20190102_Rad.nc"),
]

In [ ]:
# define ERA5 sources
era5pl = ERA5(time=time, 
              variables=Cocip.met_variables, 
              pressure_levels=pressure_levels, 
              path=met_filepaths)
era5sl = ERA5(time=time, 
              variables=Cocip.rad_variables, 
              path=rad_filepaths)

# create `MetDataset` from sources
mds = era5pl.open_metdataset()
rds = era5sl.open_metdataset()

In [ ]:
mds

In [ ]:
rds

In [ ]:
from pycontrails.models import Cocip, CocipParams

In [ ]:
# there params can be overridden via the `params` keyword in the Cocip class
? CocipParams

In [ ]:
? Cocip

In [ ]:
# use pathlib here for cross platform compatibility
# assumes `bada` folder is in the current directory
path_prefix = pathlib.Path("bada")

# these can also just be strings i.e. "bada/bada3"
bada3_path = pathlib.Path(path_prefix / "bada3")
bada4_path = pathlib.Path(path_prefix / "bada4")

In [ ]:
# the only parameters we will override here is the BADA paths
params = {
    "bada3_path": bada3_path,
    "bada4_path": bada4_path,
}

flight = flights[3] # select one flight from above

cocip = Cocip(met=mds, rad=rds, params=params)
out = cocip.eval(flight=flight)

In [ ]:
# output Flight and Flight data (pd.DataFrame)
cocip.flight.dataframe.head()

In [ ]:
# output contrail waypoints (pd.DataFrame)
cocip.contrail.head()

In [ ]:
# look at the contrail waypoint from the last 5 timesteps
cocip.contrail_list[-5:-1]

In [ ]:
# plot flight and contrail waypoints together
# contrail color indicates net radiative forcing of the contrail segment
ax = flight.dataframe.plot(x="longitude", y="latitude", color="k", legend=False, figsize=(8, 6))
_ = cocip.contrail.plot.scatter(x="longitude", y="latitude", c="rf_net", cmap="magma", ax=ax)

In [ ]:
# output contrail waypoints as xarray Dataset 
cocip.contrail_dataset